In [1]:
import torch
from torch import nn
import numpy as np
import os
import time
import codecs
import re
import json
import pandas as pd
import glob

In [4]:
#get weight data
rootdir = "MyFitbitData5/Mad/personal & Account"
#rootdir = "MyFitbitData/name/personal & Account"
regex = re.compile('weight-2021-03-05')
#regex = re.compile('weight-2021-most-recent')

filenames = os.listdir(rootdir)
weights= []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            weights = np.append(weights, file)

print(weights)            
for x in range(np.size(weights)):
    filename = rootdir + "/" + weights[x]    
    data = []
    weight = []
    fat = []
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        for x in range(np.size(roughdata)):
            del roughdata[x]["logId"]
            del roughdata[x]["time"]
            del roughdata[x]["source"]
            del roughdata[x]["bmi"]
            weight = np.append(weight, roughdata[x]["weight"])
            fat = np.append(fat, roughdata[x]["fat"])

        data = np.append(data, roughdata)
weightfat = data
print(weightfat)


['weight-2021-03-05.json']
[{'weight': 116.6, 'fat': 16.0, 'date': '03/11/21'}
 {'weight': 116.6, 'fat': 16.0, 'date': '03/12/21'}
 {'weight': 114.6, 'fat': 15.300000190734863, 'date': '03/13/21'}
 {'weight': 114.6, 'fat': 14.5, 'date': '03/14/21'}
 {'weight': 112.0, 'fat': 13.899999618530273, 'date': '03/15/21'}
 {'weight': 112.8, 'fat': 13.199999809265137, 'date': '03/16/21'}
 {'weight': 112.8, 'fat': 13.199999809265137, 'date': '03/17/21'}
 {'weight': 112.0, 'fat': 14.600000381469727, 'date': '03/18/21'}
 {'weight': 112.2, 'fat': 14.600000381469727, 'date': '03/19/21'}
 {'weight': 110.0, 'fat': 14.600000381469727, 'date': '03/20/21'}
 {'weight': 110.8, 'fat': 13.899999618530273, 'date': '03/21/21'}
 {'weight': 109.8, 'fat': 13.899999618530273, 'date': '03/22/21'}
 {'weight': 109.8, 'fat': 13.899999618530273, 'date': '03/23/21'}
 {'weight': 110.4, 'fat': 14.600000381469727, 'date': '03/24/21'}
 {'weight': 107.6, 'fat': 14.600000381469727, 'date': '03/25/21'}
 {'weight': 109.8, 'fat':

In [5]:
#get calorie data
rootdir = "MyFitbitData5/Mad/Nutrition"

regex = re.compile('food_logs-0')
filenames = os.listdir(rootdir)
foodNames = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            foodNames = np.append(foodNames, file)

start = False
#127
for x in range(np.size(foodNames)):
    filename = 'MyFitbitData5/Mad/Nutrition/' + foodNames[x]
    data = []
    count = 0
    print(filename)
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        
        for y in range(np.size(roughdata)):
            
            if roughdata[y-count]["logDate"] == "2021-03-27":
                start = True
            if roughdata[y-count]["logDate"] == "2021-03-10":
                start = False
            
            if start == True:
                cal = roughdata[y-count]["loggedFood"]["calories"]
                del roughdata[y-count]["logId"]
                del roughdata[y-count]["loggedFood"]
                del roughdata[y-count]["favorite"]
                if "nutritionalValues" in roughdata[y-count].keys():                
                    del roughdata[y-count]["nutritionalValues"]                
                roughdata[y-count]["calories"] = cal
            else:
                del roughdata[y-count]
                count +=1

                
        
        data = np.append(data, roughdata)
calories = data[0]["calories"]
totCalories = np.empty((0,1))
dates = np.empty((0,1))
dates = np.vstack((dates, [data[0]["logDate"]]))
for x in range(len(data)-1):
    if data[x+1]["logDate"] == data[x]["logDate"]:
        calories += data[x+1]["calories"]
        if x == len(data)-2:
            totCalories = np.vstack((totCalories, calories))
    else:
        dates = np.vstack((dates, data[x+1]["logDate"]))
        totCalories = np.vstack((totCalories, calories))
        print(data[x+1])
        calories = data[x+1]["calories"]
        
    

totCalories = np.flip(np.vstack((totCalories, [[1000]])))
print(totCalories)

MyFitbitData5/Mad/Nutrition/food_logs-0.json
{'logDate': '2021-03-26', 'calories': 140}
{'logDate': '2021-03-25', 'calories': 211}
{'logDate': '2021-03-24', 'calories': 177}
{'logDate': '2021-03-23', 'calories': 283}
{'logDate': '2021-03-22', 'calories': 167}
{'logDate': '2021-03-21', 'calories': 80}
{'logDate': '2021-03-20', 'calories': 226}
{'logDate': '2021-03-19', 'calories': 409}
{'logDate': '2021-03-18', 'calories': 275}
{'logDate': '2021-03-17', 'calories': 252}
{'logDate': '2021-03-16', 'calories': 342}
{'logDate': '2021-03-15', 'calories': 180}
{'logDate': '2021-03-14', 'calories': 340}
{'logDate': '2021-03-13', 'calories': 423}
{'logDate': '2021-03-12', 'calories': 266}
[[1000.]
 [ 567.]
 [1052.]
 [ 932.]
 [ 601.]
 [ 810.]
 [ 775.]
 [ 757.]
 [1021.]
 [2191.]
 [ 658.]
 [ 744.]
 [1097.]
 [ 632.]
 [ 857.]
 [1611.]
 [1261.]]


In [106]:

def caloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    for x in range(len(weight)):
        fatMass = np.append(fatMass, weight[x]*fat[x]/100)
        leanMass = np.append(leanMass, weight[x] - fatMass[x])
    calories = [] 
    print(fatMass)
    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*442.39)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*70
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*265
    calories = np.append(calories, [0])

    return calories
        
    
calories = caloriesBurned(weight, fat)
print(calories)
for x in range(len(calories)):
    calories[x] = totCalories[x] - calories[x]

#calories = np.append(calories, [2000,2000, 2000])
print(calories)

[18.656      18.656      17.53380022 16.617      15.56799957 14.88959978
 14.88959978 16.35200043 16.38120043 16.06000042 15.40119958 15.26219958
 15.26219958 16.11840042 15.70960041 14.49359979 14.34839979]
[  -0.         -557.8959766  -162.63119077 -572.6372691    91.65866162
   -0.          488.58337522   58.17978814 -273.61167113   95.13531859
 -121.76220858   -0.          360.84063081 -348.23303598  367.29365004
 -131.07102722    0.        ]
[1000.         1124.8959766  1214.63119077 1504.6372691   509.34133838
  810.          286.41662478  698.82021186 1294.61167113 2095.86468141
  779.76220858  744.          736.15936919  980.23303598  489.70634996
 1742.07102722 1261.        ]


In [9]:
#get heart rates
rootdir = "MyFitbitData5/Mad/Physical Activity"
#rootdir = "MyFitbitData/Name/Physical Activity"
regex = re.compile('heart_rate-2021-03')
filenames = os.listdir(rootdir)
heartRates = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            heartRates = np.append(heartRates, file)


#read files for hr data
inputs = np.empty((0,3))
dateTime = []
start = False
minInputs = np.empty((0,3))
hrDates = np.empty((0,1))
count = 1
for x in range(np.size(heartRates)):
    filename = 'MyFitbitData5/Mad/Physical Activity/' + heartRates[x]
    data = []
    
    with open(filename) as json_file:
        data = json.load(json_file)
        inputs = np.empty((len(data), 3))
        for y in range(len(data)):
            temp = data[y]["dateTime"][9:] 
            
            
            if data[y]["dateTime"][:8] == "03/11/21":
                start = True
                
            if data[y]["dateTime"][:8] == "03/28/21":
                start = False
            if start == True:
                inputs[y][0] = int(temp[:2])*3600  + int(temp[3:5])*60 + int(temp[6:9])
                inputs[y][1] = int(data[y]["value"]["bpm"])
                inputs[y][2] = count
            
       # hrDates = np.vstack((hrDates, data[10]["dateTime"][:8]))
        if start == True:
            count += 1
            minInputs = np.vstack((minInputs, inputs))



from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))

for x in range(len(minInputs) - 1):
    if minInputs[x][0] <= minInputs [x+1][0]:
        minInputs[x][0] = minInputs[x+1][0] - minInputs[x][0]
    else:
        minInputs[x][0] = 86400 - minInputs[x][0] + minInputs[x+1][0]
        
        
print(minInputs)    
'''if minInputs[x][0] > 10:
        tot = int(minInputs[x][0]/10)
        print(tot)
        print(minInputs[x])
        
        minInputs[x][0] = minInputs[x][0]%10
        
        for y in range(tot):    
            intervals = minInputs[0:x+y, 0:1]
            currentInterval = np.sum(intervals)           
            minInputs = np.insert(minInputs, [x+1],[currentInterval, (minInputs[x][1]+minInputs[x+1][1])/2,int((currentInterval+((tot-y)*10))/86400)], axis=0)'''

minInputs = np.delete(minInputs, len(minInputs)-1, 0)
train_data_normalized2 = scaler.fit_transform(np.transpose(minInputs)[1] .reshape(-1,1))
scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized1 = scaler.fit_transform(np.transpose(minInputs)[0] .reshape(-1,1))

scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized3 = scaler.fit_transform(np.transpose(minInputs)[2] .reshape(-1,1))

train_data_normalized = np.hstack((train_data_normalized1, train_data_normalized2))

train_data_normalized = np.hstack((train_data_normalized, train_data_normalized3))
train_data_normalized =torch.FloatTensor(train_data_normalized)
inputs = train_data_normalized.view(-1, 404, train_data_normalized.shape[1])
print(inputs)
numOfInputs = np.size(np.transpose(minInputs)[0])

[[5.0000e+00 4.2000e+01 1.0000e+00]
 [1.5000e+01 4.3000e+01 1.0000e+00]
 [5.0000e+00 4.2000e+01 1.0000e+00]
 ...
 [5.0000e+00 5.1000e+01 1.7000e+01]
 [5.0000e+00 5.0000e+01 1.7000e+01]
 [8.6389e+04 5.1000e+01 1.7000e+01]]
tensor([[[-0.9977, -0.8870, -1.0000],
         [-0.9919, -0.8757, -1.0000],
         [-0.9977, -0.8870, -1.0000],
         ...,
         [-0.9948, -0.8983, -1.0000],
         [-0.9948, -0.9096, -1.0000],
         [-0.9977, -0.9209, -1.0000]],

        [[-0.9948, -0.9322, -1.0000],
         [-0.9977, -0.9209, -1.0000],
         [-0.9977, -0.9322, -1.0000],
         ...,
         [-0.9977, -0.9322, -1.0000],
         [-0.9977, -0.9209, -1.0000],
         [-0.9919, -0.9096, -1.0000]],

        [[-0.9977, -0.9209, -1.0000],
         [-0.9977, -0.9096, -1.0000],
         [-0.9977, -0.9209, -1.0000],
         ...,
         [-0.9977, -0.8192, -1.0000],
         [-0.9977, -0.8870, -1.0000],
         [-0.9948, -0.9209, -1.0000]],

        ...,

        [[-0.9948, -0.8192,  1.0

In [11]:
y_actual = []

for x in range(numOfInputs):
    interval = minInputs[x][0]
    day = minInputs[x][2]
   
    intervalCalorie = calories[int(day-1)]/86400 * interval        
    y_actual = np.append(y_actual, intervalCalorie)


scaler = MinMaxScaler(feature_range=(-1,1))

normalized_y = torch.FloatTensor(scaler.fit_transform(y_actual .reshape(-1,1)))
outputs = normalized_y.view(-1, 404, normalized_y.shape[1])
print(normalized_y)
print(outputs)

tensor([[-0.9976],
        [-0.9924],
        [-0.9976],
        ...,
        [-0.9969],
        [-0.9969],
        [-0.9969]])
tensor([[[-0.9976],
         [-0.9924],
         [-0.9976],
         ...,
         [-0.9950],
         [-0.9950],
         [-0.9976]],

        [[-0.9950],
         [-0.9976],
         [-0.9976],
         ...,
         [-0.9976],
         [-0.9976],
         [-0.9924]],

        [[-0.9976],
         [-0.9976],
         [-0.9976],
         ...,
         [-0.9976],
         [-0.9976],
         [-0.9950]],

        ...,

        [[-0.9936],
         [-0.9969],
         [-0.9969],
         ...,
         [-0.9962],
         [-0.9936],
         [-0.9936]],

        [[-0.9904],
         [-0.9969],
         [-0.9936],
         ...,
         [-0.9936],
         [-0.9969],
         [-0.9969]],

        [[-0.9969],
         [-0.9969],
         [-0.9969],
         ...,
         [-0.9969],
         [-0.9969],
         [-0.9969]]])


In [81]:
'''class RNN(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 10, output_size = 1):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, 1)
        self.linear = nn.Linear(hidden_size, output_size)        
        
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq),1,-1), self.hidden_cell)
        predictions = self.linear(lstm_out)
        return predictions
    
n_hidden = 5
lstm = RNN()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)
print(lstm)'''

class RNN(nn.Module):
 
    def __init__(self, input_dim=3, hidden_dim=20, batch_size=404, output_dim=1,
                    num_layers=2):
        super(RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
 
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
 
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
 
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
 
    def forward(self, input_seq):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input_seq.view(len(input_seq),1,-1))     
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred
lstm = RNN()

loss_fn = torch.nn.MSELoss(size_average=False)
optimiser = torch.optim.Adam(lstm.parameters(), lr=0.001)

C:\Users\User\anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [114]:
epochs = 1000

for i in range(epochs):
    for x in range(40): 
        lstm.zero_grad()

        y_pred = lstm(inputs[x])
        
        single_loss = loss_function(y_pred.float(), outputs[x].view(404, 1, -1).float())
        loss = loss_fn(y_pred, outputs[x])
      
        # Zero out gradient, else they will accumulate between epochs
        optimiser.zero_grad()

        # Backward pass
        loss.backward()

        # Update parameters
        optimiser.step()
        
print("WOOOOO")
    
    


WOOOOO


In [112]:
num = 45
y_pred = lstm(inputs[num])
single_loss = loss_function(y_pred.float(), outputs[0].float())
yo = y_pred.detach().numpy()

woo =scaler.inverse_transform(np.transpose(np.transpose(yo)[0]))
print(np.sum(woo))
#print(woo)
outs = scaler.inverse_transform(outputs[num])
print(np.sum(outs))
print(np.hstack((woo,outs)))


56.518486
35.048713621699974
[[0.10183563 0.03905909]
 [0.13930278 0.03905909]
 [0.09947445 0.03905909]
 [0.12784605 0.01301925]
 [0.14493883 0.03905909]
 [0.15299626 0.02603984]
 [0.15765172 0.03905909]
 [0.15820022 0.02603984]
 [0.16197008 0.03905909]
 [0.1644289  0.02603984]
 [0.16923955 0.03905909]
 [0.17184019 0.02603984]
 [0.17686892 0.03905909]
 [0.17585087 0.01301925]
 [0.17654651 0.03905909]
 [0.17729433 0.03905909]
 [0.176493   0.01301925]
 [0.17827357 0.02603984]
 [0.17819466 0.03905909]
 [0.17645554 0.01301925]
 [0.18413171 0.03905909]
 [0.17893578 0.02603984]
 [0.18058124 0.03905909]
 [0.17617193 0.02603984]
 [0.17153117 0.03905909]
 [0.16824558 0.02603984]
 [0.16892652 0.01301925]
 [0.16744694 0.02603984]
 [0.16539478 0.02603984]
 [0.17034991 0.01301925]
 [0.16686366 0.02603984]
 [0.16250518 0.02603984]
 [0.1671486  0.02603984]
 [0.16324899 0.02603984]
 [0.15861493 0.01301925]
 [0.15371999 0.02603984]
 [0.15366782 0.03905909]
 [0.14912473 0.02603984]
 [0.15133341 0.026039

C:\Users\User\anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([404, 1])) that is different to the input size (torch.Size([404, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
